In [ ]:
import h5py
import numpy as np
import cv2
import os
from PIL import Image
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
data_folder = 'E:/allen'
for folder in os.listdir(data_folder):
    print(folder)

In [ ]:
data_folder = 'E:/allen'
img_data = np.empty((0,256,256,3), dtype='i8')
for folder in os.listdir(data_folder):
    class_folder = os.path.join(data_folder,folder)
    for filename in os.listdir(class_folder):
        img_path = os.path.join(class_folder,filename)
        img_raw = Image.open(img_path)
        newsize = (256, 256)
        img_scaled = img_raw.resize(newsize)
        img_arr = np.asarray(img_scaled)
        img_arr_expand = np.expand_dims(img_arr, axis=0)
        img_data = np.append(img_data, img_arr_expand, axis=0)
        print(np.shape(img_data))

In [ ]:
print(img_data.min())
print(img_data.max())
img_data_float = img_data.astype('float32')
img_data_float /= 255.0
print(img_data_float.min())
print(img_data_float.max())

In [ ]:
data_folder = 'E:/allen'
labels = []
for folder in os.listdir(data_folder):
    class_folder = os.path.join(data_folder,folder)
    for filename in os.listdir(class_folder):
        labels.append(folder)

In [ ]:
enc = OneHotEncoder(sparse=False)
label_enc = LabelEncoder()
labels_arr = np.array(labels)
y_labels_int = label_enc.fit_transform(labels_arr)
print(y_labels_int)
print(np.shape(y_labels_int))
y_labels = np.reshape(y_labels_int, (6077, 1))
print(y_labels)
y_vector = enc.fit_transform(y_labels)
print(y_vector)
print(np.shape(y_vector))

In [ ]:
images_split = np.array_split(img_data_float, 3, axis=3)
ref_channel = np.append(images_split[0], images_split[2], axis=3)
print(np.shape(ref_channel))

In [ ]:
print(type(img_data_float))
print(type(y_vector))
print(type(ref_channel))

In [ ]:
from sklearn.utils import shuffle
X_data_rand, Y_data_rand, R_data_rand = shuffle(img_data_float, y_vector, ref_channel)

In [ ]:
def store_h5_split(images, ref_channel, labels):
    """ Stores an array of images to HDF5.
        Parameters:
        ---------------
        images       images array, (N, 256, 256, 3) to be stored
        ref_channel  reference array, (N, 256, 256, 2) to be stored
        labels       labels array, (N, 10) to be stored
    """

    # Create a new HDF5 file
    file = h5py.File('allen_fa_norm.h5', "w")

    # Create a dataset in the file
    dataset_X = file.create_dataset(
        "X", np.shape(images), h5py.h5t.IEEE_F32BE, data=images
    )
    
    dataset_Y = file.create_dataset(
        "Y", np.shape(labels), h5py.h5t.STD_U8BE, data=labels
    )
    
    dataset_R = file.create_dataset(
        "R", np.shape(ref_channel), h5py.h5t.IEEE_F32BE, data=ref_channel
    )
    file.close()

In [ ]:
store_h5_split(X_data_rand, R_data_rand, Y_data_rand)